<center><h1>[CBIC]</h1></center>
<center><h1>Processando resultados</h1></center>

### Sumário:

1. [LSSVM Global](#lssvm-g)

    1.1 [Acurácia nos conjuntos de treino e teste](#lssvm-g-acuracia)

    1.2 [Análise dos autovalores da matriz de kernel](#lssvm-g-autovalores)


2. [LSSVM Local](#lssvm-l)

    2.1 [Acurácia nos conjuntos de treino e teste](#lssvm-l-acuracia)

    2.2 [Análise dos autovalores da matriz de kernel](#lssvm-l-autovalores)
    
    2.3 [Análise das métricas de validação de agrupamentos](#lssvm-l-metricas)


3. [Estudo comparativo entre LSSVM global e local](#comparativo)

    3.1 [Tabelas comparativas de treino e teste](#comparativo_tabelas)

### 1.  LSSVM Global <a class="anchor" id="lssvm-g"></a>

Carregando resultados:

In [1]:
import glob
import pandas as pd
import numpy as np

path      = "simulation_results/cbic/temp_glssvm_cbic"
all_files = glob.glob(path + "/*.csv")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df_g = pd.concat(li, axis=0, ignore_index=True)
df_g.head()

,dataset_name,random_state,$\gamma$,$\sigma$,eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
0,pk,1073219128,100.0,0.316228,b'\xeb\xb9\x93\xae\xd8\xa3\t@v}\xd6\xf6W\x16\x...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x12\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00...
1,pk,1083408183,1000000.0,133.352143,b'L\x05\x8b=@?X@\xd6\xd6?\xaaL\xday?\xef\xb4T\...,float64,b'\x12\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00...,b'\x0e\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x...
2,pk,1084850658,10000.0,133.352143,b'\xb7\x07\xdcBX?X@\xeev\xef\xc7\x0b\x89u?\t\x...,float64,b'\r\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x...,b'\x0f\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x...
3,pk,1122163231,100.0,2.371374,b'\xc3\x9b\xb3\x85\xc8(S@\xa8\xe94E/Q\x1b@B\xe...,float64,b'\x15\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00...,b'\x0f\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x...
4,pk,1150350141,100.0,2.371374,"b""\x11q\x0cX1\x8cS@\xf0\x98F \x1aC\x18@\xd0Y\x...",float64,b'\x17\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00...,b'\x0f\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x...


In [2]:
datasets = np.unique(df_g['dataset_name'].values).tolist()
datasets

['pk', 'vc2c', 'vc3c', 'wf24f', 'wf2f', 'wf4f']

## 1.1 Acurácia nos conjuntos de treino e teste <a class="anchor" id="lssvm-g-acuracia"></a>

In [3]:
from IPython.core.display import display, HTML
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True) # enabling plot within jupyter notebook

for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    
    df_dataset = df_g.loc[df_g['dataset_name'] == dataset_name]
    
    results = {'acc_train': None, 'acc_test': None}

    train_series, test_series = df_dataset['cm_tr'].values, df_dataset['cm_ts'].values

    n_exp = len(train_series)
    acc_train = [None]*n_exp
    acc_test  = [None]*n_exp
    for i in range(n_exp):
        # train accuracy
        temp_tr = np.frombuffer(eval(train_series[i]), dtype='int64')
        cm_tr = temp_tr.reshape( int( len(temp_tr)**(1/2) ) ,-1)
        acc_train[i] = np.trace(cm_tr)/np.sum(cm_tr)

        # test accuracy
        temp_ts = np.frombuffer(eval(test_series[i]), dtype='int64')
        cm_ts = temp_ts.reshape( int( len(temp_ts)**(1/2) ) ,-1)
        acc_test[i] = np.trace(cm_ts)/np.sum(cm_ts)

    results['acc_train'] = acc_train
    results['acc_test']  = acc_test

    train_box = go.Box(
        y = results['acc_train'],
        name = "treino",
        boxmean='sd'
    )

    test_box = go.Box(
        y = results['acc_test'],
        name = "teste",
        boxmean='sd'
    )

    layout = go.Layout(
        title = "Acurácia nos conjuntos de treino e teste para G-LSSVM [<b>{}</b>]".format(dataset_name),
        showlegend=True,
        yaxis=dict(title="Acurácia"),
        xaxis=dict(title="Conjunto")
    )

    fig = go.Figure(data=[train_box, test_box],layout=layout)
    fig.show()
    display(HTML('<hr>'))

C:\Users\Renan\AppData\Local\Temp\ipykernel_9268\1905791217.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
import plotly.figure_factory as ff
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True) # enabling plot within jupyter notebook

bin_sizes = {
    'pk'    : .020,
    'vc2c'  : .0125,
    'vc3c'  : .013,
    'wf24f' : .00155,
    'wf2f'  : .0009,
    'wf4f'  : .00095
}
for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    
    df_dataset = df_g.loc[df_g['dataset_name'] == dataset_name]

    results = {'acc_train': None, 'acc_test': None}
   
    train_series, test_series = df_dataset['cm_tr'].values, df_dataset['cm_ts'].values

    n_exp = len(train_series)
    acc_train = [None]*n_exp
    acc_test  = [None]*n_exp
    for i in range(n_exp):
        # train accuracy
        temp_tr = np.frombuffer(eval(train_series[i]), dtype='int64')
        cm_tr = temp_tr.reshape( int( len(temp_tr)**(1/2) ) ,-1)
        acc_train[i] = np.trace(cm_tr)/np.sum(cm_tr)

        # test accuracy
        temp_ts = np.frombuffer(eval(test_series[i]), dtype='int64')
        cm_ts = temp_ts.reshape( int( len(temp_ts)**(1/2) ) ,-1)
        acc_test[i] = np.trace(cm_ts)/np.sum(cm_ts)

    results['acc_train'] = acc_train
    results['acc_test']  = acc_test


    hist_data = [results['acc_train'], results['acc_test']]

    group_labels = ['treino', 'teste']

    # Create distplot with custom bin_size
    fig = ff.create_distplot(hist_data, group_labels, bin_size=bin_sizes[dataset_name])
    fig.update_layout(
        title_text='Histograma da acurácia nos conjuntos de treino e '+
                   'teste para o conjunto de dados <b>{}</b>'.format(dataset_name),
        xaxis_title = 'Acurácia',
        yaxis_title = 'Frequência'
    )
    fig.show()
    
    display(HTML('<hr>'))

## 1.2 Análise dos autovalores da matriz de kernel <a class="anchor" id="lssvm-g-autovalores"></a>

In [181]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML

for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    print(" ")
    # get dataframe of the specific dataset
    df_dataset = df_g.loc[df_g['dataset_name'] == dataset_name]

    hps_names = ["$\gamma$","$\sigma$"]
    hps_comb = np.unique(df_dataset[hps_names].values, axis=0) # hyperparam combinations
#     print(hps_comb)
    for comb in hps_comb:
        df_comb = df_dataset.loc[ (df_dataset[hps_names[0]]==comb[0]) & (df_dataset[hps_names[1]]==comb[1]) ]
        
#         display(df_comb)

        eigen_string = df_comb['eigenvalues'].values
        dtypes       = df_comb['eigenvalues_dtype'].values
        eigenvalues_list = [None]*len(dtypes)
        cond_list = np.empty(len(dtypes)) # conditioning
        for i in range( len(dtypes) ):
            eigenvalues_list[i] = np.frombuffer(eval(eigen_string[i]), dtype=dtypes[i])
            modules = np.absolute(eigenvalues_list[i])
            cond_list[i] = np.amax(modules)/np.amin(modules)


        freq = len(eigenvalues_list)

        eigenvalues = np.concatenate(eigenvalues_list)
        cond_worst = np.amax(cond_list)

#         x = eigenvalues.real.tolist()
#         y = eigenvalues.imag.tolist()

#         fig = go.Figure()
#         fig.add_trace(
#             go.Histogram2dContour(x=x, y=y, colorscale='Hot', reversescale=True, xaxis='x', yaxis='y')
#         )
#         fig.add_trace(
#             go.Scatter(x=x, y=y, xaxis='x', yaxis='y', mode='markers',
#                 marker=dict(color='rgba(0,0,0,0.4)', size=6)
#             ))
#         fig.add_trace(go.Histogram(y=y,
#                 xaxis = 'x2',
#                 marker = dict(
#                     color = 'rgba(0,0,0,1)'
#                 )
#             ))
#         fig.add_trace(go.Histogram(x=x,
#                 yaxis = 'y2',
#                 marker = dict(
#                     color = 'rgba(0,0,0,1)'
#                 )
#             ))


#         title = "Eigenvalues in <b>{}</b> dataset with ".format(dataset_name)
#         title+="<b>gamma={:.2E}</b>; <b>sigma={:.2E}</b>".format(comb[0], comb[1])
#         title+= " [<b>{} instances</b>]".format(freq)
#         fig.update_layout(
#             title = title,
#             xaxis = dict(zeroline=False, showgrid=False,
#                 domain = [0,0.85], 
#                 title='Real part'
#             ),
#             yaxis = dict(zeroline = False, 
#                 domain = [0,0.85],
#                 title='Imaginary part'
#             ),
#             xaxis2 = dict(zeroline = False, showgrid = False,
#                 domain = [0.85,1]
#             ),
#             yaxis2 = dict(zeroline = False, showgrid = False,
#                 domain = [0.85,1]
#             ),
#             bargap = 0,
#             hovermode = 'closest',
#             showlegend = False
#         )

#         fig.show()

        print("case: ", end='')
        print("gamma = {:.2E}; sigma = {:.2E}".format(comb[0], comb[1]), end=' ')
        print("[{:}]".format(str(freq)+" instances"))
        print("dtypes = {}".format(np.unique(dtypes)))
        print("Worst conditioning: {:.2E}".format(cond_worst))
        print("-"*55)
        
        print("\n")
        
    display(HTML('<hr>'))     
#     print("\n"+"#"*100+"\n")

 
case: gamma = 1.00E+00; sigma = 3.16E-01 [3 instances]
dtypes = ['float64']
Worst conditioning: 1.44E+02
-------------------------------------------------------


case: gamma = 1.00E+02; sigma = 3.16E-01 [17 instances]
dtypes = ['float64']
Worst conditioning: 1.21E+02
-------------------------------------------------------


case: gamma = 1.00E+02; sigma = 2.37E+00 [10 instances]
dtypes = ['float64']
Worst conditioning: 4.62E+06
-------------------------------------------------------


case: gamma = 1.00E+02; sigma = 1.78E+01 [1 instances]
dtypes = ['float64']
Worst conditioning: 2.00E+10
-------------------------------------------------------


case: gamma = 1.00E+04; sigma = 3.16E-01 [2 instances]
dtypes = ['float64']
Worst conditioning: 1.37E+02
-------------------------------------------------------


case: gamma = 1.00E+04; sigma = 2.37E+00 [4 instances]
dtypes = ['float64']
Worst conditioning: 3.49E+06
-------------------------------------------------------


case: gamma = 1.00

 
case: gamma = 1.00E+00; sigma = 3.16E-01 [13 instances]
dtypes = ['float64']
Worst conditioning: 2.53E+06
-------------------------------------------------------


case: gamma = 1.00E+00; sigma = 2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 3.17E+13
-------------------------------------------------------


case: gamma = 1.00E+02; sigma = 3.16E-01 [4 instances]
dtypes = ['float64']
Worst conditioning: 1.72E+06
-------------------------------------------------------


case: gamma = 1.00E+02; sigma = 2.37E+00 [10 instances]
dtypes = ['float64']
Worst conditioning: 1.42E+15
-------------------------------------------------------


case: gamma = 1.00E+04; sigma = 2.37E+00 [4 instances]
dtypes = ['float64']
Worst conditioning: 1.71E+15
-------------------------------------------------------


case: gamma = 1.00E+04; sigma = 1.33E+02 [1 instances]
dtypes = ['complex128']
Worst conditioning: 3.42E+18
-------------------------------------------------------


case: gamma = 1

 
case: gamma = 1.00E+00; sigma = 3.16E-01 [1 instances]
dtypes = ['float64']
Worst conditioning: 8.28E+05
-------------------------------------------------------


case: gamma = 1.00E+00; sigma = 2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 4.18E+13
-------------------------------------------------------


case: gamma = 1.00E+02; sigma = 3.16E-01 [1 instances]
dtypes = ['float64']
Worst conditioning: 7.12E+04
-------------------------------------------------------


case: gamma = 1.00E+02; sigma = 2.37E+00 [11 instances]
dtypes = ['float64']
Worst conditioning: 2.41E+15
-------------------------------------------------------


case: gamma = 1.00E+02; sigma = 1.78E+01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 2.35E+18
-------------------------------------------------------


case: gamma = 1.00E+04; sigma = 2.37E+00 [12 instances]
dtypes = ['float64']
Worst conditioning: 5.38E+15
-------------------------------------------------------


case: gamma = 1

 
case: gamma = 1.00E+02; sigma = 2.37E+00 [49 instances]
dtypes = ['float64']
Worst conditioning: 7.27E+12
-------------------------------------------------------


case: gamma = 1.00E+06; sigma = 1.78E+01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 9.74E+18
-------------------------------------------------------




 
case: gamma = 1.00E+04; sigma = 3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 3.57E+34
-------------------------------------------------------


case: gamma = 1.00E+06; sigma = 3.16E-01 [49 instances]
dtypes = ['complex128']
Worst conditioning: 4.76E+35
-------------------------------------------------------




 
case: gamma = 1.00E+02; sigma = 3.16E-01 [3 instances]
dtypes = ['complex128']
Worst conditioning: 7.51E+34
-------------------------------------------------------


case: gamma = 1.00E+04; sigma = 3.16E-01 [39 instances]
dtypes = ['complex128']
Worst conditioning: 7.68E+34
-------------------------------------------------------


case: gamma = 1.00E+06; sigma = 3.16E-01 [8 instances]
dtypes = ['complex128']
Worst conditioning: 1.36E+35
-------------------------------------------------------




# 2. LSSVM Local <a class="anchor" id="lssvm-l"></a>

Carregando resultados:

In [182]:
import pandas as pd
import numpy as np

# df = pd.read_csv('./simulation_results/G-LSSVM - n_init=50 - 2019-08-28 12:58:37.036007.csv')
# df.head()

In [183]:
import glob

path = r'./temp_llssvm_cbic/results'
all_files = glob.glob(path + "/*.csv")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df_l = pd.concat(li, axis=0, ignore_index=True)
df_l.head()

,dataset_name,random_state,# empty regions,# homogeneous regions,$\gamma_{opt}$ [CV],$\sigma_{opt}$ [CV],$\gamma_{opt}$ [Adjusted Rand Index],$\sigma_{opt}$ [Adjusted Rand Index],$\gamma_{opt}$ [Adjusted Mutual Information],$\sigma_{opt}$ [Adjusted Mutual Information],...,cv_score [Davies-Bouldin],cv_score [Dunn],cv_score [Final Prediction Error],cv_score [Akaike Information Criteria],cv_score [Bayesian Information Criteria],cv_score [Minimum Description Length],eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
0,vc2c,553198502,0,2,10000.0,17.782794,10000.0,2.371374,10000.0,2.371374,...,0.706775,0.715150,0.806452,0.806452,0.684041,0.685036,b'\xe2\x7f\x83\xf2}\xfeA@`\xeb*\xc6O\xefs?\xfe...,float64,b'a\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x0...,b'W\x00\x00\x00\x00\x00\x00\x00\x12\x00\x00\x0...
1,pk,1769740991,0,3,10000.0,2.371374,1000000.0,17.782794,1000000.0,17.782794,...,0.737233,0.775077,0.737233,0.737233,0.737233,0.737233,b'/\x9d\xdc\xe1\xa3\x8d1@Y\xf1\x8c>\x08\x1b\xe...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
2,wf2f,1984592806,0,27,10000.0,0.316228,1000000.0,0.316228,1000000.0,0.316228,...,0.947008,0.955912,0.972733,0.972733,0.972733,0.972733,b'\x8ey\xcc\xea\xc8\xb1L@\x00\x00\x00\x00\x00\...,complex128,b'7\x04\x00\x00\x00\x00\x00\x00\x06\x00\x00\x0...,b'1\x04\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x0...
3,wf24f,1369999811,0,0,100.0,2.371374,100.0,2.371374,100.0,2.371374,...,0.832226,0.836784,0.850447,0.850447,0.871909,0.866319,b'\xa3\xd9]\x80fh\x86@\x12\xc6\x8e\x00\x16\x17...,float64,b'5\x04\x00\x00\x00\x00\x00\x00\x12\x00\x00\x0...,b'\xd2\x03\x00\x00\x00\x00\x00\x00T\x00\x00\x0...
4,pk,780297093,0,0,10000.0,0.316228,1000000.0,1000.000000,10000.0,0.316228,...,0.797905,0.818093,0.881913,0.881913,0.881913,0.881913,b'\xbf\x14\xf0\x91\x8fw\x02@\xf5>\xcc|\xc6\xf5...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x10\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00...


## 2.1 Acurácia nos conjuntos de treino e teste <a class="anchor" id="lssvm-l-acuracia"></a>

In [184]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True) # enabling plot within jupyter notebook

for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    
    df_dataset = df_l.loc[df_l['dataset_name'] == dataset_name]
    
    results = {'acc_train': None, 'acc_test': None}
    train_series, test_series = df_dataset['cm_tr'].values, df_dataset['cm_ts'].values

    n_exp = len(train_series)
    acc_train = [None]*n_exp
    acc_test  = [None]*n_exp
    for i in range(n_exp):
        # train accuracy
        temp_tr = np.frombuffer(eval(train_series[i]), dtype='int64')
        cm_tr = temp_tr.reshape( int( len(temp_tr)**(1/2) ) ,-1)
        acc_train[i] = np.trace(cm_tr)/np.sum(cm_tr)

        # test accuracy
        temp_ts = np.frombuffer(eval(test_series[i]), dtype='int64')
        cm_ts = temp_ts.reshape( int( len(temp_ts)**(1/2) ) ,-1)
        acc_test[i] = np.trace(cm_ts)/np.sum(cm_ts)

    results['acc_train'] = acc_train
    results['acc_test']  = acc_test

    train_box = go.Box(
        y = results['acc_train'],
        name = "treino",
        boxmean='sd'
    )

    test_box = go.Box(
        y = results['acc_test'],
        name = "teste",
        boxmean='sd'
    )

    layout = go.Layout(
        title = "Acurácia nos conjuntos de treino e teste para o L-LSSVM [<b>{}</b>]".format(dataset_name),
        showlegend=True,
        yaxis=dict(title="Acurácia"),
        xaxis=dict(title="Conjunto")
    )

    fig = go.Figure(data=[train_box, test_box],layout=layout)
    fig.show()
    
    display(HTML('<hr>'))

In [185]:
import plotly.figure_factory as ff
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True) # enabling plot within jupyter notebook

bin_sizes = {
    'pk'    : .015,
    'vc2c'  : .015,
    'vc3c'  : .0145,
    'wf24f' : .0015,
    'wf2f'  : .00072,
    'wf4f'  : .0015
}
for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    
    df_dataset = df_l.loc[df_l['dataset_name'] == dataset_name]

    results = {'acc_train': None, 'acc_test': None}
   
    train_series, test_series = df_dataset['cm_tr'].values, df_dataset['cm_ts'].values

    n_exp = len(train_series)
    acc_train = [None]*n_exp
    acc_test  = [None]*n_exp
    for i in range(n_exp):
        # train accuracy
        temp_tr = np.frombuffer(eval(train_series[i]), dtype='int64')
        cm_tr = temp_tr.reshape( int( len(temp_tr)**(1/2) ) ,-1)
        acc_train[i] = np.trace(cm_tr)/np.sum(cm_tr)

        # test accuracy
        temp_ts = np.frombuffer(eval(test_series[i]), dtype='int64')
        cm_ts = temp_ts.reshape( int( len(temp_ts)**(1/2) ) ,-1)
        acc_test[i] = np.trace(cm_ts)/np.sum(cm_ts)

    results['acc_train'] = acc_train
    results['acc_test']  = acc_test

    hist_data = [results['acc_train'], results['acc_test']]

    group_labels = ['treino', 'teste']

    # Create distplot with custom bin_size
    fig = ff.create_distplot(hist_data, group_labels, bin_size=bin_sizes[dataset_name])
    fig.update_layout(
        title = "Acurácia nos conjuntos de treino e teste para L-LSSVM [<b>{}</b>]".format(dataset_name),
        xaxis_title = 'Acurácia',
        yaxis_title = 'Frequência'
    )
    fig.show()
    
    display(HTML('<hr>'))

## 2.2 Análise dos autovalores da matriz de kernel <a class="anchor" id="lssvm-l-autovalores"></a>

In [149]:
import plotly.graph_objects as go
import itertools
from IPython.core.display import display, HTML

for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    print(" ")
    # get dataframe of the specific dataset
    df_dataset = df.loc[df['dataset_name'] == dataset_name]

    hps_names = ["$k_{opt}$ [CV]","$\gamma_{opt}$ [CV]","$\sigma_{opt}$ [CV]"]
    hps_comb = np.unique(df_dataset[hps_names].values, axis=0) # hyperparam combinations
    for comb in hps_comb:
        df_comb = df_dataset.loc[(df_dataset[hps_names[0]]==comb[0]) & 
                                 (df_dataset[hps_names[1]]==comb[1]) &
                                 (df_dataset[hps_names[2]]==comb[2]) 
                                ]
        
        
        eigen_string = df_comb['eigenvalues'].values
        dtypes       = df_comb['eigenvalues_dtype'].values
        eigenvalues_list = [None]*len(dtypes)
        cond_list = []#np.empty(len(dtypes)) # conditioning
        for i in range( len(dtypes) ):
            eigvals_full = np.frombuffer(eval(eigen_string[i]), 
                                     dtype=dtypes[i])
            
            nan_indices = np.argwhere(np.isnan(eigvals_full))
            eigvals_list = [None]*len(nan_indices)
            last_nan=-1
            for j in range(len(nan_indices)):
            #     print(nan_indices[i][0])
                eigvals_list[j] = eigvals_full[last_nan+1:nan_indices[j][0]]
                last_nan=nan_indices[j][0]
                
                modules = np.absolute(eigvals_list[j])
                cond_list.append(np.amax(modules)/np.amin(modules))
            
            eigenvalues_list[i] = eigvals_list
        
        
        freq = len(eigenvalues_list)
        
        # list of lists to single list
        merged = list(itertools.chain(*eigenvalues_list)) 
        
        eigenvalues = np.concatenate(merged)
        cond_worst = np.amax(cond_list)

#         x = eigenvalues.real.tolist()
#         y = eigenvalues.imag.tolist()

#         fig = go.Figure()
#         fig.add_trace(
#             go.Histogram2dContour(x=x, y=y, colorscale='Hot', reversescale=True, xaxis='x', yaxis='y')
#         )
#         fig.add_trace(
#             go.Scatter(x=x, y=y, xaxis='x', yaxis='y', mode='markers',
#                 marker=dict(color='rgba(0,0,0,0.4)', size=6)
#             ))
#         fig.add_trace(go.Histogram(y=y,
#                 xaxis = 'x2',
#                 marker = dict(
#                     color = 'rgba(0,0,0,1)'
#                 )
#             ))
#         fig.add_trace(go.Histogram(x=x,
#                 yaxis = 'y2',
#                 marker = dict(
#                     color = 'rgba(0,0,0,1)'
#                 )
#             ))


#         title = "Eigenvalues in <b>{}</b> dataset with ".format(dataset_name)
#         title+="<b>k_opt={}; gamma={:.2E}; sigma={:.2E}</b>".format(comb[0], comb[1], comb[2])
#         title+= " [<b>{} instances</b>]".format(freq)
#         fig.update_layout(
#             title = title,
#             xaxis = dict(zeroline=False, showgrid=False,
#                 domain = [0,0.85], 
#                 title='Real part'
#             ),
#             yaxis = dict(zeroline = False, 
#                 domain = [0,0.85],
#                 title='Imaginary part'
#             ),
#             xaxis2 = dict(zeroline = False, showgrid = False,
#                 domain = [0.85,1]
#             ),
#             yaxis2 = dict(zeroline = False, showgrid = False,
#                 domain = [0.85,1]
#             ),
#             bargap = 0,
#             hovermode = 'closest',
#             showlegend = False
#         )

#         fig.show()

        print("case: ", end='')
        print("k_opt={}; gamma={:.2E}; sigma={:.2E}".format(int(comb[0]), comb[1], comb[2]), end=' ')
        print("[{:}]".format(str(freq)+" instances"))
        print("dtypes = {}".format(np.unique(dtypes)))
        print("Worst conditioning: {:.2E}".format(cond_worst))
        print("-"*55)
        
        print("\n")
    
    display(HTML('<hr>'))
#     print("\n"+"#"*100+"\n")

 
case: k_opt=2; gamma=1.00E+00; sigma=3.16E-01 [2 instances]
dtypes = ['float64']
Worst conditioning: 1.44E+02
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=3.16E-01 [4 instances]
dtypes = ['float64']
Worst conditioning: 7.17E+01
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=2.37E+00 [3 instances]
dtypes = ['float64']
Worst conditioning: 3.43E+06
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=3.16E-01 [1 instances]
dtypes = ['float64']
Worst conditioning: 3.47E+01
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=1.78E+01 [1 instances]
dtypes = ['float64']
Worst conditioning: 1.36E+09
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+06; sigma=1.78E+01 [1 instances]
dtypes = ['float64']
Worst conditioning: 2.09E+10
------------------------------------------------

 
case: k_opt=2; gamma=1.00E+00; sigma=3.16E-01 [5 instances]
dtypes = ['float64']
Worst conditioning: 6.37E+05
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=2.37E+00 [3 instances]
dtypes = ['float64']
Worst conditioning: 2.64E+13
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=2.37E+00 [2 instances]
dtypes = ['float64']
Worst conditioning: 2.68E+13
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=1.78E+01 [4 instances]
dtypes = ['complex128']
Worst conditioning: 8.68E+18
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+06; sigma=2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 1.67E+13
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+06; sigma=1.78E+01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 8.88E+18
------------------------------------------

 
case: k_opt=2; gamma=1.00E+02; sigma=3.16E-01 [1 instances]
dtypes = ['float64']
Worst conditioning: 6.37E+04
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=2.37E+00 [2 instances]
dtypes = ['float64']
Worst conditioning: 1.60E+13
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=1.78E+01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 5.45E+18
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=2.37E+00 [6 instances]
dtypes = ['float64']
Worst conditioning: 8.53E+13
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=1.78E+01 [2 instances]
dtypes = ['complex128']
Worst conditioning: 9.16E+18
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+06; sigma=1.78E+01 [7 instances]
dtypes = ['complex128']
Worst conditioning: 2.87E+21
---------------------------------------

 
case: k_opt=2; gamma=1.00E+02; sigma=2.37E+00 [16 instances]
dtypes = ['float64']
Worst conditioning: 3.44E+12
-------------------------------------------------------


case: k_opt=3; gamma=1.00E+02; sigma=2.37E+00 [25 instances]
dtypes = ['float64']
Worst conditioning: 2.17E+12
-------------------------------------------------------


case: k_opt=3; gamma=1.00E+06; sigma=1.78E+01 [1 instances]
dtypes = ['float64']
Worst conditioning: 1.03E+17
-------------------------------------------------------


case: k_opt=4; gamma=1.00E+02; sigma=2.37E+00 [4 instances]
dtypes = ['float64']
Worst conditioning: 3.25E+11
-------------------------------------------------------


case: k_opt=4; gamma=1.00E+06; sigma=1.78E+01 [1 instances]
dtypes = ['float64']
Worst conditioning: 4.86E+16
-------------------------------------------------------


case: k_opt=7; gamma=1.00E+02; sigma=2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 1.70E+11
----------------------------------------------

 
case: k_opt=5; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 2.30E+34
-------------------------------------------------------


case: k_opt=6; gamma=1.00E+06; sigma=3.16E-01 [2 instances]
dtypes = ['complex128']
Worst conditioning: 1.05E+35
-------------------------------------------------------


case: k_opt=11; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 1.56E+35
-------------------------------------------------------


case: k_opt=13; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 6.66E+33
-------------------------------------------------------


case: k_opt=19; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 4.30E+68
-------------------------------------------------------


case: k_opt=20; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 8.28E+34
--------------------------

/home/romulo/Dropbox/Mestrado/Pesquisa/Local_Modeling/venv/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning:

divide by zero encountered in double_scalars



 
case: k_opt=2; gamma=1.00E+04; sigma=3.16E-01 [10 instances]
dtypes = ['complex128']
Worst conditioning: 1.10E+34
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 6.65E+30
-------------------------------------------------------


case: k_opt=5; gamma=1.00E+04; sigma=3.16E-01 [2 instances]
dtypes = ['complex128']
Worst conditioning: 6.26E+33
-------------------------------------------------------


case: k_opt=6; gamma=1.00E+04; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 8.03E+33
-------------------------------------------------------


case: k_opt=7; gamma=1.00E+04; sigma=3.16E-01 [3 instances]
dtypes = ['complex128']
Worst conditioning: 8.60E+34
-------------------------------------------------------


case: k_opt=8; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 1.66E+31
-----------------------------

## 2.3 Análise das métricas de validação de agrupamentos <a class="anchor" id="lssvm-l-metricas"></a>

In [155]:
# pegando o nome das métricas e sua quantidade
n_metrics = int((len(df.columns[6:-4])-1)/4)

metric_names = [' ']*n_metrics
count=0
for text in list(df.columns[-4-n_metrics:-4]):
    metric_names[count] = text[10:-1]
    count+=1

print("Quantidade de métrica: {}".format(len(metric_names)))

Quantidade de métrica: 12


In [193]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML

for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    
    # get dataframe of the specific dataset
    df_dataset = df_l.loc[df_l['dataset_name'] == dataset_name]
        
    # boxplot to k_{opt}
    fig = go.Figure(data=[go.Histogram(x=df_dataset["$k_{opt}$ [CV]"].values.tolist(),
                                      xbins_size=.5)])
    fig.update_layout(
        title = "Distribuição do k_opt para as {} rodadas no conjunto <b>{}</b>".format(
            len(df_dataset),
            dataset_name),
        xaxis_title = 'Número de agrupamentos ótimo',
        yaxis_title = 'Frequência'
    )
    fig.show()
    
    
    x = [i+1 for i in range(len(df_dataset))]
    # Create traces
    fig = go.Figure()
    for name in metric_names:
        column_name = "cv_score [{}]".format(name)
        fig.add_trace(go.Scatter(
            x=x, 
            y=df_dataset[column_name].values.tolist(),
            mode='lines+markers',
#             mode='markers',
            name=name))
    
    fig.update_layout(
        title_text='Pontuação na validação cruzada da proposta de cada métrica de validação de agrupamentos,'+
                   '<br>para cada rodada no conjunto <b>{}</b>'.format(dataset_name),
        yaxis_title = 'Pontuação na validação cruzada',
        xaxis_title = 'Rodada #'
    )
    fig.show()
    
    # frequência de proposta ótima para cada métrica
    
#     import plotly.graph_objects as go
#     animals=['giraffes', 'orangutans', 'monkeys']

#     fig = go.Figure([go.Bar(x=animals, y=[20, 14, 23])])
#     fig.update_layout(
#         title_text='Frequência de proposta ótima para cada métrica,'+
#                    '<br>para cada rodada no conjunto <b>{}</b>'.format(dataset_name),
#         yaxis_title = 'Pontuação na validação cruzada',
#         xaxis_title = 'Rodada #'
#     )
#     fig.show()
    
    display(HTML("<hr>"))
    
#     lista = list(df_dataset.columns)[2:-4]
#     for name in lista:
#         display(df_dataset[name].value_counts())

# 3.  Estudo comparativo entre LSSVM global e local <a class="anchor" id="comparativo"></a>

## 3.1 Tabelas comparativas de treino e teste <a class="anchor" id="comparativo_tabelas"></a>

In [199]:
for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    
    df_g_dataset = df_g.loc[df_g['dataset_name'] == dataset_name]
    df_l_dataset = df_l.loc[df_l['dataset_name'] == dataset_name]
        
    results = {
        'global': { 
            'acc_train': None, 
            'acc_test': None,
            
            
        },
        'local': {}
    }

    train_series, test_series = df_dataset['cm_tr'].values, df_dataset['cm_ts'].values

    n_exp = len(train_series)
    acc_train = [None]*n_exp
    acc_test  = [None]*n_exp
    for i in range(n_exp):
        # train accuracy
        temp_tr = np.frombuffer(eval(train_series[i]), dtype='int64')
        cm_tr = temp_tr.reshape( int( len(temp_tr)**(1/2) ) ,-1)
        acc_train[i] = np.trace(cm_tr)/np.sum(cm_tr)

        # test accuracy
        temp_ts = np.frombuffer(eval(test_series[i]), dtype='int64')
        cm_ts = temp_ts.reshape( int( len(temp_ts)**(1/2) ) ,-1)
        acc_test[i] = np.trace(cm_ts)/np.sum(cm_ts)

    results['acc_train'] = acc_train
    results['acc_test']  = acc_test

    train_box = go.Box(
        y = results['acc_train'],
        name = "treino",
        boxmean='sd'
    )

    test_box = go.Box(
        y = results['acc_test'],
        name = "teste",
        boxmean='sd'
    )

    layout = go.Layout(
        title = "Acurácia nos conjuntos de treino e teste para G-LSSVM [<b>{}</b>]".format(dataset_name),
        showlegend=True,
        yaxis=dict(title="Acurácia"),
        xaxis=dict(title="Conjunto")
    )

    fig = go.Figure(data=[train_box, test_box],layout=layout)
    fig.show()
    display(HTML('<hr>'))

,dataset_name,random_state,$\gamma$,$\sigma$,eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
0,pk,771721097,100.0,0.316228,b'\x18\xb4@\x1b\n\xee\n@\x1a\r\x83E\x19d\x06@\...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x10\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00...
1,pk,661309350,100.0,0.316228,b'\xfcz\xbb4c\xd5\x0c@PA\x94\x82% \t@\xbe\xc2\...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\r\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x...
2,pk,73470257,10000.0,17.782794,b'\xdbtz$\x02\x1fX@yd\xc0\xfd\x8c\x06\xd2?&s\x...,float64,b'\r\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x...,b'\r\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x...
3,pk,769056302,1000000.0,1000.000000,b'\x00SN\x1c\xfd?X@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'\x0c\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00...,b'\x0f\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x...
12,pk,888866443,100.0,2.371374,b'r\xb4\xa7\xc3I\x04S@\x93hI]9\x82\x1e@z\x08\x...,float64,b'\x14\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00...,b'\x11\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00...


,dataset_name,random_state,# empty regions,# homogeneous regions,$\gamma_{opt}$ [CV],$\sigma_{opt}$ [CV],$\gamma_{opt}$ [Adjusted Rand Index],$\sigma_{opt}$ [Adjusted Rand Index],$\gamma_{opt}$ [Adjusted Mutual Information],$\sigma_{opt}$ [Adjusted Mutual Information],...,cv_score [Davies-Bouldin],cv_score [Dunn],cv_score [Final Prediction Error],cv_score [Akaike Information Criteria],cv_score [Bayesian Information Criteria],cv_score [Minimum Description Length],eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
1,pk,1769740991,0,3,10000.0,2.371374,1000000.0,17.782794,1000000.0,17.782794,...,0.737233,0.775077,0.737233,0.737233,0.737233,0.737233,b'/\x9d\xdc\xe1\xa3\x8d1@Y\xf1\x8c>\x08\x1b\xe...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
4,pk,780297093,0,0,10000.0,0.316228,1000000.0,1000.000000,10000.0,0.316228,...,0.797905,0.818093,0.881913,0.881913,0.881913,0.881913,b'\xbf\x14\xf0\x91\x8fw\x02@\xf5>\xcc|\xc6\xf5...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x10\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00...
6,pk,1813743798,0,2,1000000.0,17.782794,1000000.0,17.782794,1000000.0,17.782794,...,0.808520,0.730120,0.808520,0.808520,0.808520,0.808520,b'd)\x87\x82S\xf0G@\xfca\rZ\xe1\x01\xa7?\xef!\...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x14\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00...
10,pk,2105288516,0,1,10000.0,0.316228,100.0,0.316228,1000000.0,2.371374,...,0.823401,0.888846,0.823401,0.823401,0.823401,0.823401,"b'.x^\xff|\x1a\xfd?\xba\x15\xf5fT,\xd2?\\\x83\...",float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x0f\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x...
12,pk,661309350,0,4,100.0,2.371374,100.0,2.371374,1000000.0,17.782794,...,0.831863,0.899781,0.831863,0.831863,0.831863,0.831863,b'\xc3\x18\x92c\\9(@;\xb0\xd8:G>\xe1?x\xc8=%\x...,float64,b'\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x13\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00...


,dataset_name,random_state,$\gamma$,$\sigma$,eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
6,vc2c,496715263,100.0,2.371374,b'^\x19\'P\xb4Vb@q*\xea\xa2\xa6\xb9\x12@\xc8!\...,float64,b'c\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x0...,"b""c\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x0..."
8,vc2c,1616248553,100.0,0.316228,b'\xe2\x85i\x873\x1c?@r\xbc.T\xfb\x010@\xc7\x0...,float64,b'h\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x0...,"b""W\x00\x00\x00\x00\x00\x00\x00\x12\x00\x00\x0..."
21,vc2c,658101020,1000000.0,17.782794,b'\xf7d$N\xc1Zc@\x00\x00\x00\x00\x00\x00\x00\x...,complex128,b'`\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\...,b'X\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x0...
22,vc2c,1369999811,1000000.0,2.371374,b'\x0cylXvrb@\xf9zZ\x03\x98\xc6\x0c@+\x1b{|\xc...,float64,b'a\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x0...,b'U\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x0...
26,vc2c,1515536585,10000.0,2.371374,b'\xf0\xdd\xbb\xd6\x95hb@9%\xf6\xfd\xb3\x1e\x1...,float64,b'a\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x0...,b'_\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\...


,dataset_name,random_state,# empty regions,# homogeneous regions,$\gamma_{opt}$ [CV],$\sigma_{opt}$ [CV],$\gamma_{opt}$ [Adjusted Rand Index],$\sigma_{opt}$ [Adjusted Rand Index],$\gamma_{opt}$ [Adjusted Mutual Information],$\sigma_{opt}$ [Adjusted Mutual Information],...,cv_score [Davies-Bouldin],cv_score [Dunn],cv_score [Final Prediction Error],cv_score [Akaike Information Criteria],cv_score [Bayesian Information Criteria],cv_score [Minimum Description Length],eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
0,vc2c,553198502,0,2,10000.0,17.782794,10000.0,2.371374,10000.0,2.371374,...,0.706775,0.715150,0.806452,0.806452,0.684041,0.685036,b'\xe2\x7f\x83\xf2}\xfeA@`\xeb*\xc6O\xefs?\xfe...,float64,b'a\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x0...,b'W\x00\x00\x00\x00\x00\x00\x00\x12\x00\x00\x0...
7,vc2c,773164400,0,2,1000000.0,17.782794,1000000.0,133.352143,1000000.0,133.352143,...,0.755558,0.755558,0.755558,0.755558,0.720060,0.755558,"b'""\xe9\xe2X\x1d\xfe+@\xf8{\x826\x9eO`?R\xa4\x...",float64,b'c\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x0...,b'X\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x0...
14,vc2c,771721097,0,3,1000000.0,17.782794,1.0,0.316228,1.0,0.316228,...,0.832552,0.832552,0.761506,0.851613,0.829140,0.761506,b'\xb8\x0f\x86\r\xf7\xfdH@\xe6\xfc45\xb4\xb9{?...,float64,b'a\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x0...,b'X\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x0...
17,vc2c,933374443,0,1,10000.0,17.782794,1000000.0,133.352143,1000000.0,133.352143,...,0.792271,0.733390,0.751721,0.751721,0.797906,0.748747,b'\xdcg\x1b\x89\xd8\xbeP@\x00\x00\x00\x00\x00\...,complex128,b'a\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x0...,b'_\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\...
22,vc2c,658101020,0,0,10000.0,17.782794,10000.0,17.782794,10000.0,17.782794,...,0.800000,0.774194,0.774194,0.774194,0.755558,0.753561,b'\xf3\x13y\x9dP\xfdO@\x00\x00\x00\x00\x00\x00...,complex128,b'a\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x0...,b'Y\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x0...


,dataset_name,random_state,$\gamma$,$\sigma$,eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
17,vc3c,1341250263,10000.0,133.352143,b'\xe5N%\x12\xe8_c@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'\n\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x...,b'\x0c\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00...
23,vc3c,1948415885,100.0,2.371374,b'\x95\x8c=6\xe2\xeaa@\xb0\x9a\x7f\xc8~\xe8\x1...,float64,b'\x19\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00...,b'\x13\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00...
25,vc3c,496715263,10000.0,2.371374,"b'8,[\xbe\xca\xfda@yn/\x90/\xdf\x18@2,\x83\xd1...",float64,b'\x17\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00...,"b""\x15\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00..."
29,vc3c,972975441,10000.0,133.352143,b'\xa6\xe0\xb0\xcc\xe0_c@\x00\x00\x00\x00\x00\...,complex128,b'\x0e\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00...,b'\x13\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00...
30,vc3c,1717607707,10000.0,17.782794,b'\xa9\x1f\x1c\xa3\xdbXc@\x00\x00\x00\x00\x00\...,complex128,b'\x15\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x...,b'\x14\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x...


...

,dataset_name,random_state,# empty regions,# homogeneous regions,$\gamma_{opt}$ [CV],$\sigma_{opt}$ [CV],$\gamma_{opt}$ [Adjusted Rand Index],$\sigma_{opt}$ [Adjusted Rand Index],$\gamma_{opt}$ [Adjusted Mutual Information],$\sigma_{opt}$ [Adjusted Mutual Information],...,cv_score [Davies-Bouldin],cv_score [Dunn],cv_score [Final Prediction Error],cv_score [Akaike Information Criteria],cv_score [Bayesian Information Criteria],cv_score [Minimum Description Length],eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
3,wf24f,1369999811,0,0,100.0,2.371374,100.0,2.371374,100.0,2.371374,...,0.832226,0.836784,0.850447,0.850447,0.871909,0.866319,b'\xa3\xd9]\x80fh\x86@\x12\xc6\x8e\x00\x16\x17...,float64,b'5\x04\x00\x00\x00\x00\x00\x00\x12\x00\x00\x0...,b'\xd2\x03\x00\x00\x00\x00\x00\x00T\x00\x00\x0...
25,wf24f,1934644852,0,0,100.0,2.371374,100.0,2.371374,100.0,2.371374,...,0.853989,0.862305,0.864763,0.864763,0.874182,0.863363,b'\x07\x18i\x0bj\x15\x87@\xe7J\xeb\x07\x89HZ@\...,float64,b'8\x04\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x0...,b'\xd0\x03\x00\x00\x00\x00\x00\x00R\x00\x00\x0...
34,wf24f,553198502,0,0,100.0,2.371374,100.0,2.371374,100.0,2.371374,...,0.842657,0.842657,0.862730,0.862730,0.857145,0.857020,b'\x16\x82G\x9a\xe6\xf3\x85@*\x8f\\\xe2\xd1gY@...,float64,b'0\x04\x00\x00\x00\x00\x00\x00\x11\x00\x00\x0...,b'\xef\x03\x00\x00\x00\x00\x00\x00@\x00\x00\x0...
36,wf24f,1545156723,0,0,100.0,2.371374,100.0,2.371374,100.0,2.371374,...,0.841413,0.840413,0.849048,0.849048,0.868514,0.863087,b'F=r!\x10\x8bt@J@+Za\xd1>@\x9c\xbc +^;7@pSb\x...,float64,b'3\x04\x00\x00\x00\x00\x00\x00\x14\x00\x00\x0...,b'\xdc\x03\x00\x00\x00\x00\x00\x00K\x00\x00\x0...
37,wf24f,439954366,0,0,100.0,2.371374,1000000.0,17.782794,10000.0,17.782794,...,0.834349,0.823034,0.827101,0.827101,0.851262,0.846789,b'\xb9B\x90\xd0\x9a#\x87@\x8c\x8bW?3\xfbZ@1\x0...,float64,b'4\x04\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x0...,b'\xdb\x03\x00\x00\x00\x00\x00\x003\x00\x00\x0...


,dataset_name,random_state,$\gamma$,$\sigma$,eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
5,wf2f,1340173848,1000000.0,0.316228,b'\\\xadw\x84jf\xa0@\x00\x00\x00\x00\x00\x00\x...,complex128,"b'\xff\x03\x00\x00\x00\x00\x00\x00""\x00\x00\x0...",b'\x08\x04\x00\x00\x00\x00\x00\x00\x1a\x00\x00...
9,wf2f,1467373767,1000000.0,0.316228,b'\xb6\xe5\x9f\xb1\xa7m\xa0@\x00\x00\x00\x00\x...,complex128,b'\x05\x04\x00\x00\x00\x00\x00\x00\x1e\x00\x00...,b'\x06\x04\x00\x00\x00\x00\x00\x00\x1d\x00\x00...
11,wf2f,769056302,1000000.0,0.316228,b'\xe9\xe2\xa4\xbel\x88\xa0@\x00\x00\x00\x00\x...,complex128,b'\x04\x04\x00\x00\x00\x00\x00\x00 \x00\x00\x0...,b'\x0c\x04\x00\x00\x00\x00\x00\x00\x1a\x00\x00...
27,wf2f,383776909,1000000.0,0.316228,b'\xe0\xb0B\xde\xf8\xa7\xa0@\x00\x00\x00\x00\x...,complex128,b'\x02\x04\x00\x00\x00\x00\x00\x00$\x00\x00\x0...,b'\t\x04\x00\x00\x00\x00\x00\x00\x1e\x00\x00\x...
44,wf2f,888866443,1000000.0,0.316228,b'\x03\x80\r)gr\xa0@\x00\x00\x00\x00\x00\x00\x...,complex128,b'\n\x04\x00\x00\x00\x00\x00\x00\x1a\x00\x00\x...,b'\xfe\x03\x00\x00\x00\x00\x00\x00(\x00\x00\x0...


,dataset_name,random_state,# empty regions,# homogeneous regions,$\gamma_{opt}$ [CV],$\sigma_{opt}$ [CV],$\gamma_{opt}$ [Adjusted Rand Index],$\sigma_{opt}$ [Adjusted Rand Index],$\gamma_{opt}$ [Adjusted Mutual Information],$\sigma_{opt}$ [Adjusted Mutual Information],...,cv_score [Davies-Bouldin],cv_score [Dunn],cv_score [Final Prediction Error],cv_score [Akaike Information Criteria],cv_score [Bayesian Information Criteria],cv_score [Minimum Description Length],eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
2,wf2f,1984592806,0,27,10000.0,0.316228,1000000.0,0.316228,1000000.0,0.316228,...,0.947008,0.955912,0.972733,0.972733,0.972733,0.972733,b'\x8ey\xcc\xea\xc8\xb1L@\x00\x00\x00\x00\x00\...,complex128,b'7\x04\x00\x00\x00\x00\x00\x00\x06\x00\x00\x0...,b'1\x04\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x0...
5,wf2f,991487909,0,20,1000000.0,0.316228,1000000.0,0.316228,1000000.0,0.316228,...,0.958893,0.961503,0.970465,0.970465,0.970465,0.970465,b'\x19\r\xcc\x84\x87\xd32@\x00\x00\x00\x00\x00...,complex128,b'?\x04\x00\x00\x00\x00\x00\x00\x05\x00\x00\x0...,b'<\x04\x00\x00\x00\x00\x00\x00\x07\x00\x00\x0...
8,wf2f,1415319480,0,8,10000.0,0.316228,1000000.0,0.316228,1000000.0,0.316228,...,0.968365,0.963917,0.963881,0.963881,0.963881,0.963881,b'\xdc\x91\xee\x82\xd3\xc5f@\x00\x00\x00\x00\x...,complex128,b'1\x04\x00\x00\x00\x00\x00\x00\r\x00\x00\x00\...,b'.\x04\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\...
15,wf2f,496715263,0,26,1000000.0,2.371374,1000000.0,0.316228,1000000.0,0.316228,...,0.938863,0.958057,0.966474,0.966474,0.966474,0.966474,b'f\xbe\xdb\xde\xbb\xfdR@\x00\x00\x00\x00\x00\...,complex128,b'.\x04\x00\x00\x00\x00\x00\x00\x10\x00\x00\x0...,b':\x04\x00\x00\x00\x00\x00\x00\x07\x00\x00\x0...
16,wf2f,2105288516,0,7,1000000.0,0.316228,1000000.0,0.316228,1000000.0,0.316228,...,0.962361,0.968109,0.974899,0.974899,0.974899,0.974899,b'kny\xbb\x84\xaaV@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'7\x04\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\...,b'3\x04\x00\x00\x00\x00\x00\x00\x04\x00\x00\x0...


,dataset_name,random_state,$\gamma$,$\sigma$,eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
13,wf4f,1934644852,10000.0,0.316228,b'3m\xfe|V\xed\x94@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'\x1f\x04\x00\x00\x00\x00\x00\x00\x16\x00\x00...,b'\x07\x04\x00\x00\x00\x00\x00\x00(\x00\x00\x0...
15,wf4f,1948415885,10000.0,0.316228,"b'9\xce\x03\xfa,\x11\x95@\x00\x00\x00\x00\x00\...",complex128,b'\x15\x04\x00\x00\x00\x00\x00\x00 \x00\x00\x0...,b'\x1e\x04\x00\x00\x00\x00\x00\x00\x1f\x00\x00...
19,wf4f,1369999811,10000.0,0.316228,b'rkB\x12@\xfa\x94@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'\x1b\x04\x00\x00\x00\x00\x00\x00\x1a\x00\x00...,"b""\x10\x04\x00\x00\x00\x00\x00\x00'\x00\x00\x0..."
24,wf4f,1545156723,10000.0,0.316228,b'\xbfQq\x17:(\x95@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'\x18\x04\x00\x00\x00\x00\x00\x00\x1f\x00\x00...,b'\xfc\x03\x00\x00\x00\x00\x00\x00(\x00\x00\x0...
34,wf4f,1084850658,10000.0,0.316228,"b',UM\xceM\xcd\x94@\x00\x00\x00\x00\x00\x00\x0...",complex128,b'\x1c\x04\x00\x00\x00\x00\x00\x00\x1a\x00\x00...,b'\x12\x04\x00\x00\x00\x00\x00\x00%\x00\x00\x0...


,dataset_name,random_state,# empty regions,# homogeneous regions,$\gamma_{opt}$ [CV],$\sigma_{opt}$ [CV],$\gamma_{opt}$ [Adjusted Rand Index],$\sigma_{opt}$ [Adjusted Rand Index],$\gamma_{opt}$ [Adjusted Mutual Information],$\sigma_{opt}$ [Adjusted Mutual Information],...,cv_score [Davies-Bouldin],cv_score [Dunn],cv_score [Final Prediction Error],cv_score [Akaike Information Criteria],cv_score [Bayesian Information Criteria],cv_score [Minimum Description Length],eigenvalues,eigenvalues_dtype,cm_tr,cm_ts
9,wf4f,1616248553,0,0,10000.0,0.316228,10000.0,0.316228,10000.0,0.316228,...,0.945090,0.947181,0.945002,0.945002,0.945002,0.945002,b'\x1am;\xe9s\xd2x@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'&\x04\x00\x00\x00\x00\x00\x00\x18\x00\x00\x0...,b'\x0f\x04\x00\x00\x00\x00\x00\x00-\x00\x00\x0...
11,wf4f,439954366,0,3,1000000.0,2.371374,10000.0,0.316228,10000.0,0.316228,...,0.943895,0.940621,0.947313,0.947313,0.942668,0.942668,b'|K~\x91\xfbcU@\x00\x00\x00\x00\x00\x00\x00\x...,complex128,b'+\x04\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x0...,b'\x13\x04\x00\x00\x00\x00\x00\x00\x1a\x00\x00...
13,wf4f,1083408183,0,6,10000.0,0.316228,100.0,0.316228,10000.0,0.316228,...,0.943097,0.948634,0.941568,0.941568,0.941568,0.941568,b'\xf5gO5\xa6\x01@@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'5\x04\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x0...,b'\x19\x04\x00\x00\x00\x00\x00\x00\x1d\x00\x00...
18,wf4f,933374443,0,1,100.0,0.316228,10000.0,0.316228,10000.0,0.316228,...,0.951263,0.943575,0.950097,0.950097,0.950097,0.950097,b'rQ\xbbZ\x80\xd1U@\x00\x00\x00\x00\x00\x00\x0...,complex128,b'\x1e\x04\x00\x00\x00\x00\x00\x00\x19\x00\x00...,b'\x16\x04\x00\x00\x00\x00\x00\x00\x1b\x00\x00...
19,wf4f,383776909,0,0,10000.0,0.316228,10000.0,0.316228,10000.0,0.316228,...,0.950405,0.951532,0.942815,0.942815,0.942815,0.942815,b'r\xeb\x07T\x9b\xf5\x90@\x00\x00\x00\x00\x00\...,complex128,b'\x19\x04\x00\x00\x00\x00\x00\x00 \x00\x00\x0...,b'\x12\x04\x00\x00\x00\x00\x00\x00%\x00\x00\x0...
